In [1]:
import numpy as np
from keras.datasets import imdb
num = 2000

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Preprocessed IMDB dataset with frequency as parameter for classifying words
(X_train, Y_train), (X_test, Y_test) = imdb.load_data(num_words=num)

In [3]:
X_train

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 1920, 2, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]),
       list([1, 194, 1153, 194, 2, 78, 228, 5, 6, 1463, 2, 2, 134, 26, 4, 715, 8, 118, 1634

In [53]:
Y_train

array([1, 0, 0, ..., 0, 1, 0])

In [34]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding , Dense , LSTM , Conv1D , Flatten , Dropout

In [8]:
#All sequences padded to the same lenght
X_train = pad_sequences(X_train , padding = 'pre')

max_len = X_train.shape[1]
features = 30

#Test Sequences padded same as Train Sequences 
X_test = pad_sequences(X_test, padding = 'pre' , maxlen = max_len )

In [9]:
#building an LSTM based model . Takes less time to train while running on cpu
import tensorflow as tf
with tf.device('/cpu:0'):
    model = Sequential()
    model.add(Embedding(num ,features , input_length = max_len ))
    model.add(LSTM(50 , dropout = 0.2 , return_sequences = False ))
    model.add(Dense(1 , activation = 'sigmoid'))

In [10]:
with tf.device('/cpu:0'):
    model.summary()
    model.compile(loss = 'binary_crossentropy' , optimizer = 'adam', metrics = ['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 2494, 30)          60000     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                16200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 76,251
Trainable params: 76,251
Non-trainable params: 0
_________________________________________________________________


In [21]:
with tf.device('/cpu:0'):
    model.fit(X_train , Y_train , epochs = 1 , batch_size = 300 , verbose =1 )

Epoch 1/1
25000/25000 [==============================] - 254s 10ms/step - loss: 0.3050 - acc: 0.8770


In [54]:
#evaluating our LSTM model to test on test data

final = model.evaluate(X_test,Y_test , verbose = 1)

25000/25000 [==============================] - 153s 6ms/step


In [55]:
print("ACCURACY : " , final[1]*100)

ACCURACY :  84.96000000000001


In [48]:
#Building a CNN base model to test the performance of CNN on text data

model1 = Sequential()
model1.add(Embedding(num ,features , input_length = max_len ))
model1.add(Conv1D(128 , 3 , activation = 'relu' ))
model1.add(Conv1D(64 , 3 , activation = 'relu' ))
model1.add(Conv1D(64 , 3 , activation = 'relu' ))
model1.add(Flatten())
model1.add(Dense(1 , activation = 'relu' ))

In [49]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 2494, 30)          60000     
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 2492, 128)         11648     
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 2490, 64)          24640     
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 2488, 64)          12352     
_________________________________________________________________
flatten_6 (Flatten)          (None, 159232)            0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 159233    
Total params: 267,873
Trainable params: 267,873
Non-trainable params: 0
_________________________________________________________________


In [50]:
model1.compile(loss = 'binary_crossentropy' , optimizer = 'adam', metrics = ['accuracy'])
model1.fit(X_train , Y_train , epochs = 7 , batch_size = 300 , verbose =1 )

Epoch 1/7
25000/25000 [==============================] - 22s 887us/step - loss: 0.7445 - acc: 0.5096
Epoch 2/7
25000/25000 [==============================] - 21s 838us/step - loss: 0.6000 - acc: 0.6717
Epoch 3/7
25000/25000 [==============================] - 21s 846us/step - loss: 0.4911 - acc: 0.7706
Epoch 4/7
25000/25000 [==============================] - 22s 873us/step - loss: 0.4453 - acc: 0.7826
Epoch 5/7
25000/25000 [==============================] - 22s 871us/step - loss: 0.6163 - acc: 0.7396
Epoch 6/7
25000/25000 [==============================] - 21s 853us/step - loss: 0.4357 - acc: 0.7934
Epoch 7/7
25000/25000 [==============================] - 22s 870us/step - loss: 0.4082 - acc: 0.7889


In [52]:
#Test CNN
final = model1.evaluate(X_test[:10000],Y_test[:10000] , verbose = 1)
print("ACCURACY : " , final[1]*100)

10000/10000 [==============================] - 4s 389us/step
ACCURACY :  76.11
